In [1]:
import pandas as pd
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *
%matplotlib inline

In [2]:
q = '''select
            sum(amount) as volume,
            date_trunc('month', date) as month
        from transactions
            where status='A' and date>='2019-01-01'
        group by date_trunc('month', date)'''
monthly = redshift_query_read(q)

In [6]:
for _, r in monthly.iterrows():
    print("{:%m-%Y}: ${:,.2f}".format(r['month'], r['volume']))

04-2019: $15,688,736.94
05-2019: $16,397,207.77
06-2019: $13,362,425.45
08-2019: $14,318,646.32
10-2019: $16,730,428.52
01-2020: $13,583,707.32
04-2020: $26,769,412.27
06-2020: $216,498,413.57
01-2019: $10,763,733.18
02-2019: $12,210,571.59
03-2019: $14,725,113.35
07-2019: $11,298,414.03
09-2019: $15,919,153.31
11-2019: $15,201,002.53
12-2019: $35,525,001.01
02-2020: $15,967,177.95
03-2020: $18,697,187.43
05-2020: $20,483,573.24
07-2020: $15,184,649.11
08-2020: $16,463,498.76
09-2020: $16,362,543.09


In [17]:
q = "select id, amount, form, org, date, status, recurring from transactions where status='A' and date>='2020-01-01'"
trans = redshift_query_read(q)
trans['date'] = pd.to_datetime(trans['date'])
trans['month'] = trans['date'].dt.to_period('M')

In [18]:
trans.groupby('month')['amount'].agg(['count', 'sum']).reset_index()

,month,count,sum
0,2020-01,138643,1.358371e+07
1,2020-02,169082,1.596718e+07
2,2020-03,147539,1.869719e+07
3,2020-04,197843,2.676941e+07
4,2020-05,170698,2.048357e+07
5,2020-06,148645,2.164984e+08
6,2020-07,141787,1.518465e+07
7,2020-08,158699,1.646350e+07
8,2020-09,118484,1.636254e+07


In [19]:
trans[trans['month']=='2020-06'].sort_values('amount', ascending=False).head()

,id,amount,form,org,date,status,recurring,month


In [21]:
trans.sort_values('amount', ascending=True).tail(10)

,id,amount,form,org,date,status,recurring,month
1254302,10115215,60000.00,928052,441556,2020-08-27,A,0,2020-08
1343898,10284494,99999.00,1197,1214,2020-09-14,A,1114795,2020-09
1347749,10284495,99999.00,1197,1214,2020-09-14,A,1114795,2020-09
60025,8656420,125000.00,730178,402435,2020-01-13,A,0,2020-01
1357348,10289481,900000.00,1197,1214,2020-09-14,A,1114881,2020-09
1349702,10289482,900000.00,1197,1214,2020-09-14,A,1114881,2020-09
1332875,10274118,997500.00,1197,1214,2020-09-13,A,1114666,2020-09
1332863,10273991,999999.00,1197,1214,2020-09-13,A,1114663,2020-09
903524,9696176,99999999.99,837733,430165,2020-06-16,A,1099724,2020-06
910133,9706505,99999999.99,837733,430165,2020-06-18,A,1099969,2020-06


In [24]:
trans[(trans['date']>='2020-06-01')&(trans['date']<'2020-07-01')].sort_values('amount', ascending=False).head(10)

,id,amount,form,org,date,status,recurring,month
903524,9696176,99999999.99,837733,430165,2020-06-16,A,1099724,2020-06
910133,9706505,99999999.99,837733,430165,2020-06-18,A,1099969,2020-06
956378,9762658,50000.00,99400,31758,2020-06-29,A,0,2020-06
879404,9668209,50000.00,930036,442052,2020-06-11,A,0,2020-06
941998,9731275,26000.00,125011,38906,2020-06-23,A,0,2020-06
839462,9623280,25000.00,937213,442932,2020-06-03,A,0,2020-06
914415,9716844,25000.00,1502,1519,2020-06-20,A,0,2020-06
951939,9750980,25000.00,125012,38907,2020-06-26,A,0,2020-06
831437,9607638,21200.00,936852,442558,2020-06-01,A,0,2020-06
891845,9690702,21000.00,9244,6064,2020-06-15,A,0,2020-06
